In [1247]:
import numpy as np 
import pandas as pd
np.random.seed(42)

In [1248]:
dados = pd.read_csv("dadoss.csv", delimiter=";", decimal=",")
dados = dados[["SAPR4", "ELET6","VIVT3", "MGLU3"]]
retornos = dados.pct_change().dropna()
cov = np.array(retornos.cov())
e_r = np.array(retornos.mean()).reshape(4, 1)

In [ ]:
def timing(func): 
    import time      
    def wrapper(*args, **kwargs):          
        t = time.clock()          
        res = func(*args, **kwargs)          
        print(func.__name__, time.clock()-t)         
        return res      
    return wrapper 

### Tentativa de Minimizar a carteira usando algoritmos Genéticos. Não funcionou bem.

In [1270]:
#@timing
def minimize_gen(rp, er, cov):
    n = er.shape[0]
    
    def chromossome(n):
        x = np.random.rand(n)
        return x/sum(x)
    
    def fitness_function(w, e_r=e_r, cov=cov, rp=rp, alpha = 10, beta = 1000):
        w = w.reshape(n, 1)
        return (w.T@cov@w + beta*(w.T@e_r -rp)**2)[0][0]
    
    def select_elite(population):
        i = np.argmin([fitness_function(_) for _ in population])
        return population[i]
    
    def new_population(population):
        new_population = []
        intermediate_population=[]
    
        elite = select_elite(population)
    
        for candidate in population:
            c1, c2 = population[np.random.randint(0, len(population))][:], population[np.random.randint(0, len(population))][:]
            ic = candidate +1*(c1-c2)
        
        for j in range(len(candidate)):
            ic[j] = np.random.choice([elite[j], ic[j]], size = 1, p=[0.9, 0.1])
            
        ic = np.where(ic>=0, ic, 0)
        
        ic = ic/sum(ic)
        
        if fitness_function(ic) <= fitness_function(candidate):
            new_population.append(ic)
        else: 
            new_population.append(candidate)
    
        return new_population
    
    
    population_size = 100
    population = [chromossome(_) for _ in range(100)]
    for i in range(50):
        population = new_population(population)

    return(select_elite(population).reshape(n))